## Ejercicios plotly

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

from datetime import datetime

**ref**: _https://www.kaggle.com/datasets/matthieugimbert/french-bakery-daily-sales_

### Ejercicio 0:
- Carga el DataFrame **`Bakery sales.csv`**.

In [ ]:
df = pd.read_csv("Bakery sales.csv")

df

### Ejercicio 1:
- Transforma la columna **`unit_price`** a `float`.


- Transforma la columna **`date`** a `datetime`.


- Crea las columnas **`weekday`**, **`month`**, **`year`** y **`hour`**. Las columnas **`weekday`** y **`month`** deben estar representados por nombre, no por número.

In [ ]:
# unit_price a float

df["unit_price"] = df["unit_price"].apply(lambda x : float(x.split(" ")[0].replace(",", ".")))

In [ ]:
# date a datetime

df["date"] = df["date"].apply(lambda x : datetime.strptime(x, "%Y-%m-%d"))

In [ ]:
# weekday, month, year y hour

df["weekday"] = df["date"].apply(lambda x : x.strftime("%A"))

df["month"] = df["date"].apply(lambda x : x.strftime("%B"))

df["year"] = df["date"].apply(lambda x : x.year)

df["hour"] = df["time"].apply(lambda x : float(x.split(":")[0]))

In [ ]:
df

### Ejercicio 2:

¿A que hora se venden más productos?

- Grafica un histograma usando la columna de `hour` y `quantity`.
- Agrega el parámetro `color = year` y `nbins = 50`.

In [ ]:
px.histogram(data_frame = df,
             x          = "hour",
             y          = "quantity",
             color      = "year",
             nbins      = 50)

### Ejercicio 3:
- De la gráfica anterior quita el filtro por color y separa la gráfica en dos, usando el parámetro `facet_col`.
- Agrega a esa gráfica un violin plot usando el parámetro `marginal`.

In [ ]:
px.histogram(data_frame = df,
             x          = "hour",
             y          = "quantity",
             nbins      = 50,
             marginal   = "violin",
             facet_col  = "year")

### Ejercicio 4:

- Filtra el **DataFrame** por las filas donde la columna `article` tenga la palabra **"BAGUETTE"**.
    - ```html
      df[df["article"].str.contains("")]
      ```
    
- Haz una gráfica de **`sunburst`**, separando por año y mes, mostrando la cantidad de elementos vendidos usando el filtro de **"BAGUETTE"**.

In [ ]:
px.sunburst(data_frame = df[df["article"].str.contains("BAGUETTE")],
            values     = "quantity",
            path       = ["year", "month"])

### Ejercicio 5:
- Calcula el precio total de cada fila, multiplicando las columnas `quantity` y `unit_price`. Crea una nueva columna `total_article_price` con esta información.

- Elimina las filas donde `quantity` tiene un outlier.

- Haz un violin plot con esa nueva columna.

In [ ]:
df["total_article_price"] = df["quantity"]*df["unit_price"]

df

In [ ]:
df = df[df["quantity"].between(0, df["quantity"].quantile(q = 0.6))].reset_index(drop = True)

df.head()

In [ ]:
df.shape

In [ ]:
px.violin(data_frame = df,
                   x = "total_article_price")

### Ejercicio 6:
- Agrupa el **DataFrame** por fecha y obten la suma de la columna **`total_article_price`**. Crea un **DataFrame** nuevo.
- Busca la forma de quedarte con la columna **`weekday`** durante la agrupación.
- Haz un line plot con este **DataFrame**, utiliza también la columna **`weekday`**.

In [ ]:
df2 = df.groupby("date", as_index = False).agg({"total_article_price" : "sum",
                                                "weekday"             : "max"})
df2

In [ ]:
px.line(data_frame = df2,
        x          = "date",
        y          = "total_article_price",
        color      = "weekday")

### Ejercicio 7:
- Crea un nuevo DataFrame agrupando por la columna **`ticket_number`**, que tenga como columnas la suma de **`quantity`**, **`total_article_price`** y el maximo de **`year`**. Renombra las columnas.

- Haz un box plot usando la columnas **`quantity`** y **`year`**.

In [ ]:
df2 = df.groupby(by = "ticket_number", as_index = False).agg({"quantity"            : ["sum"],
                                                              "total_article_price" : ["sum"],
                                                              "year"                : ["max"]})

df2.columns = [x[0] for x in df2.columns]

df2

In [ ]:
px.box(data_frame = df2,
       x          = "year",
       y          = "quantity")

### Ejercicio 8:
- Haz un pie plot mostrando el total vendido por día de la semana.
- Haz un pie plot mostrando el total vendido por mes.

In [ ]:
px.pie(data_frame = df,
       names      = "weekday",
       values     = "total_article_price")

In [ ]:
px.pie(data_frame = df,
       names      = "month",
       values     = "total_article_price")

### Ejercicio 9:
- Agrupa los datos por la columna **`article`** y crea un nuevo DataFrame, donde las columnas sean la suma de **`quantity`** y **`total_article_price`** y la media de **`unit_price`** y el mínimo/máximo de **`year`**.
- Encuentra cuales son los 20 elementos más vendidos y muestralos en un bar plot, usa la columna **`total_article_price`** con el parámetro `color`.

In [ ]:
df3 = df.groupby(by = "article", as_index = False).agg({"quantity"            : ["sum"],
                                                        "total_article_price" : ["sum"],
                                                        "unit_price"          : ["mean"],
                                                        "year"                : ["min"]})

df3.columns = [x[0] for x in df3]

df3

In [ ]:
px.bar(data_frame = df3.sort_values(by = "quantity", ascending = False).head(20),
       x          = "article",
       y          = "quantity",
       color      = "total_article_price")

### Ejercicio 10:
- Usando el DataFrame, crea una nueva columna **`article_new`**, si el artículo de esa fila no está entre los 20 más vendidos entonces que tenga el nombre de **`OTRO`**, si el artículo está si entre los 20 más vendidos que no se cambie el nombre.

- Repite el bar plot anterior, esta vez usando todas las filas.

In [ ]:
mas_vendidos = df3.sort_values(by = "article", ascending = False).head(20)["article"].values

In [ ]:
df3["article_new"] = df3["article"].apply(lambda x : x if x in mas_vendidos else "OTRO")

df3

In [ ]:
px.bar(data_frame = df3,
       x          = "article_new",
       y          = "quantity",
       color      = "total_article_price")

In [ ]:
################################################################################################################################